In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
import MeshCat as mc 
# using JLD2
using Test
using Random
import Convex as cvx 
import ECOS 
using ProgressMeter

  Activating environment at `~/devel/hw_ideas/hw2/Project.toml`


In [2]:
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

animate_quadrotor (generic function with 1 method)

## Linearization warmup 
Before we apply convex MPC to nonlinear systems, let's go over what it means to "linearize our system". Specifically, we are going to approximate our nonlinear system with a first-order Taylor series, and define a new set of $(\Delta x, \Delta u)$ coordinates.

First, assume our discrete time dynamics are the following:

$$ x_{k+1} = f(x_k,u_k) $$

And we are going to linearize about a reference trajectory $\bar{x}_{1:N}, \bar{u}_{1:N-1}$. From here, we can define our delta's accordingly:

$$ \begin{align} x_k &= \bar{x}_k + \Delta x_k \\ u_k &= \bar{u}_k + \Delta u_k \end{align}$$  

Next, we are going to approximate our discrete time dynamics function with the following first order Taylor series:

$$ 
x_{k+1} \approx f(\bar{x}_k, \bar{u}_k) + \bigg[\frac{\partial f}{\partial x} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg](x_k - \bar{x}_k) + \bigg[\frac{\partial f}{\partial u} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg](u_k - \bar{u}_k)
$$

Which we can substitute in our delta notation to get the following:

$$ 
\bar{x}_{k+1} + \Delta x_{k+1} \approx f(\bar{x}_k, \bar{u}_k) + \bigg[\frac{\partial f}{\partial x} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg]\Delta x_k + \bigg[\frac{\partial f}{\partial u} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg] \Delta u_k
$$

If the trajectory $\bar{x},\bar{u}$ is dynamically feasible (meaning $\bar{x}_{k+1} = f(\bar{x}_k, \bar{u}_k)$), then we can cancel these equivalent terms on each side of the above equation, resulting in the following:

$$ 
 \Delta x_{k+1} \approx  \bigg[\frac{\partial f}{\partial x} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg]\Delta x_k + \bigg[\frac{\partial f}{\partial u} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg] \Delta u_k
$$

## Quadrotor MPC
We can formulate this MPC problem where we optimize over $x,u$:
$$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ (x_i - x_{i,ref})^TQ(x_i - x_{i,ref}) + (u_i - u_{i,ref})^TR(u_i - u_{i,ref}) \bigg] + \frac{1}{2}(x_N - x_{N,ref})^TQ_f(x_N - x_{N,ref}) & \\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_{k+1} = f(\bar{x}_k, \bar{u}_k) + \bigg[\frac{\partial f}{\partial x} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg](x_k - \bar{x}_k) + \bigg[\frac{\partial f}{\partial u} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg](u_k - \bar{u}_k) \quad &\text{for } i = 1,2,\ldots,N-1 \\ 
 & u_{min} \leq u_i \leq u_{max} \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 \end{align}$$
 Or equivalently where we optimize over $\Delta x, \Delta u$, by simply substituting in $x = \bar{x} + \Delta x$ and $u = \bar{u} + \Delta u$:
 $$ \begin{align} \min_{\Delta x_{1:N},\Delta u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ (\bar{x}_i + \Delta x_i - x_{i,ref})^TQ(\bar{x}_i + \Delta x_i - x_{i,ref}) + (\bar{u}_i + \Delta u_i - u_{i,ref})^TR(\bar{u}_i + \Delta u_i - u_{i,ref}) \bigg] \\ & +  \frac{1}{2}(\bar{x}_N + \Delta x_N - x_{N,ref})^TQ_f(\bar{x}_N + \Delta x_N - x_{N,ref}) & \\ 
 \text{st} \quad & \bar{x}_1 + \Delta x_1 = x_{\text{IC}} \\ 
 & \bar{x}_{k+1} + \Delta x_{k+1} = f(\bar{x}_k, \bar{u}_k) + \bigg[\frac{\partial f}{\partial x} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg]\Delta x_k + \bigg[\frac{\partial f}{\partial u} \bigg|_{\bar{x}_k, \bar{u}_k}  \bigg] \Delta u_k \quad &\text{for } i = 1,2,\ldots,N-1 \\ 
 & u_{min} \leq \bar{u}_i + \Delta u_i \leq u_{max} \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 \end{align}$$

In [3]:
function get_jacobians(model, X̄, Ū)
    N_mpc = length(X̄)
    A = [FD.jacobian(_x -> rk4(model,dynamics,_x,Ū[i],model.dt), X̄[i]) for i = 1:(N_mpc - 1)]
    B = [FD.jacobian(_u -> rk4(model,dynamics,X̄[i],_u,model.dt), Ū[i]) for i = 1:(N_mpc - 1)]
    return A, B
end
function convex_mpc_controller_full(model,params,x0,idx)

    N_mpc, Q, R = params.N_mpc, params.Q, params.R

    # get slice of the relevant trajectories for the N_mpc window
    X̄    =    params.X̄[idx:(idx + N_mpc - 1)]
    Ū    =    params.Ū[idx:(idx + N_mpc - 2)]
    Xref = params.Xref[idx:(idx + N_mpc - 1)]
    Uref = params.Uref[idx:(idx + N_mpc - 2)]

    # create variables 
    X = cvx.Variable(params.nx,N_mpc)
    U = cvx.Variable(params.nu,N_mpc - 1)

    # cost function (tracking cost on Xref, Uref)
    cost = 0.0
    for i = 1:N_mpc
        cost += 0.5*cvx.quadform(X[:,i] - Xref[i], Q)
    end
    for i = 1:(N_mpc - 1)
        cost += 0.5*cvx.quadform(U[:,i] - Uref[i], R)
    end
    prob = cvx.minimize(cost)

    # initial condition constraint
    prob.constraints += X[:,1] == x0

    # dynamics constraints
    A,B = get_jacobians(model, X̄, Ū)
    for i = 1:(N_mpc-1)
        prob.constraints += X[:,i+1] == rk4(model, dynamics, X̄[i], Ū[i], params.dt) + A[i]*(X[:,i] - X̄[i]) + B[i]*(U[:,i] - Ū[i])
    end

    cvx.solve!(prob, ECOS.Optimizer; silent_solver = true)

    U = U.value

    return vec(U[:,1])
end
function convex_mpc_controller_deltas(model,params,x0,idx)

    N_mpc, Q, R = params.N_mpc, params.Q, params.R

    X̄ =       params.X̄[idx:(idx + N_mpc - 1)]
    Ū =       params.Ū[idx:(idx + N_mpc - 2)]
    Xref = params.Xref[idx:(idx + N_mpc - 1)]
    Uref = params.Uref[idx:(idx + N_mpc - 2)]


    ΔX = cvx.Variable(params.nx,N_mpc)
    ΔU = cvx.Variable(params.nu,N_mpc - 1)


    cost = 0.0
    for i = 1:N_mpc
        xi = X̄[i] + ΔX[:,i]
        cost += 0.5*cvx.quadform(xi - Xref[i], Q)
    end
    for i = 1:(N_mpc - 1)
        ui = Ū[i] + ΔU[:,i]
        cost += 0.5*cvx.quadform(ui - Uref[i], R)
    end

    prob = cvx.minimize(cost)

    # initial condition constraint
    prob.constraints += X̄[1] + ΔX[:,1] == x0

    # add dynamics constraints
    A,B = get_jacobians(model, X̄, Ū)
    for i = 1:(N_mpc-1)
        prob.constraints += X̄[i+1] + ΔX[:,i+1] == rk4(model, dynamics, X̄[i], Ū[i], params.dt) + A[i]*ΔX[:,i] + B[i]*ΔU[:,i]
    end
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = true)
    ΔU = ΔU.value

    return Ū[1] + ΔU[:,1]
end

convex_mpc_controller_deltas (generic function with 1 method)

In [5]:
let

    
    # dynamics parameters
    nx = 12
    nu = 4
    N = 250
    dt = 0.1
    x0 = [0;0;1.2;0;0;0.0;zeros(6)]

    X̄ = [deepcopy(x0) for i = 1:N]
    Ū = [(9.81*0.5/4)*ones(nu) for i = 1:(N-1)]
    Q = 10*diagm(ones(nx))
    R = .1*diagm(ones(nu))

    model = (mass=0.5,
            J=Diagonal([0.0023, 0.0023, 0.004]),
            gravity=[0,0,-9.81],
            L=0.1750,
            kf=1.0,
            km=0.0245,dt = dt)

    Xref = [ [5*cos(t);5*cos(t)*sin(t);1.2;zeros(9)] for t = range(-pi/2,3*pi/2 + 4*pi, length = N)]
    for i = 1:(N-1)
        Xref[i][4:6] = (Xref[i+1][1:3] - Xref[i][1:3])/dt
    end
    Uref = deepcopy(Ū)

    N_mpc = 40

    u_min = zeros(nu)
    u_max = 10*ones(nu)
    x_min = -1e3*ones(nx)
    x_max = 1e3*ones(nx)

    params = (N = N, dt = dt, Q = Q, R = R, X̄ = X̄, Ū = Ū, Xref = Xref, Uref = Uref, N_mpc = N_mpc,#idx = idx,
    x_min = x_min, x_max = x_max, u_min = u_min, u_max = u_max,nx = nx, nu = nu)

    # main simulation
    N_sim = 100
    @assert N_sim < N
    Xsim = [deepcopy(x0) for i = 1:N_sim]
    Usim = [zeros(2) for i = 1:(N_sim-1)]
    @showprogress "simulating" for i = 1:(N_sim-1)
        Usim[i] = convex_mpc_controller_deltas(model,params,Xsim[i],i)

        # simulate
        Xsim[i+1] = rk4(model, dynamics, Xsim[i], Usim[i], dt)

    end

    display(animate_quadrotor(Xsim, Xref, params.dt))
end

simulating 100%|█████████████████████████████████████████| Time: 0:00:05
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8813


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("traj" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("s23" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x32, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s14" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x31, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s18" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x31, 0x38], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p82" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x38, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s44" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x34, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p43" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x34, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s53" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x35, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s19" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x31, 0x39], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "p60" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x70, 0x36, 0x30], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "s52" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x74, 0x72, 0x61, 0x6a, 0x2f, 0x73, 0x35, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}())…)), "vic" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x76, 0x69, 0x63], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "target" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x74, 0x2f, 0x74, 0x61, 0x72, 0x67, 0x65, 0x74], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{Any}(), ip"127.0.0.1", 8813))